In [1]:
from newspaper import Article
import numpy as np  
import newspaper

## get urls and keywords from file

In [2]:
with open('urls.txt', 'r') as f:
    urls = f.readlines()
urls = [url.replace('\n', '') for url in urls]
urls

['https://vnexpress.net/', 'https://www.24h.com.vn/', 'https://tuoitre.vn/']

In [3]:
with open('keywords.txt', 'r') as f:
    keywords = f.readlines()
keywords = [key.replace('\n', '') for key in keywords]
keywords

['thủ tướng', 'chính phủ', 'Phạm Minh Chính']

### standardize keywords

In [4]:
keywords = [key.replace(' ', '_') for key in keywords]
keywords = [key.lower() for key in keywords]
list_key_tmp = []
for key in keywords:
    list_key_tmp += key.split('_')

# keywords += list_key_tmp
keywords

['thủ_tướng', 'chính_phủ', 'phạm_minh_chính']

## crawl function

In [5]:
from bs4 import BeautifulSoup
import requests
import re  

def is_valid(url):
    # return re.findall(r'\.[a-z]{3}', url)
    return re.findall(r'\.[a-z]{3}', url) and re.search(r'https://', url)

# global dict
dict_url = {}

def get_link_articles_from_url(url):
    respone = requests.get(url)
    soup = BeautifulSoup(respone.text, 'html.parser')
    
    for a in soup.find_all('a', href=True):
        articles_link = a['href'].replace('#box_comment_vne', '')
        articles_link = articles_link.replace('#box_comment', '')
       
        if articles_link not in dict_url.keys() and is_valid(articles_link):
            dict_url[articles_link] = 1

    return list(dict_url.keys())

# test 
tmp = get_link_articles_from_url(urls[1])
print(len(tmp))
print(*tmp[:5], sep='\n')


274
https://www.24h.com.vn/bong-da-c48.html
https://www.24h.com.vn/kinh-doanh-c161.html
https://www.24h.com.vn/thi-truong-tieu-dung-c52.html
https://www.24h.com.vn/suc-khoe-doi-song-c62.html
https://www.24h.com.vn/thoi-trang-hi-tech-c407.html


In [6]:
import newspaperedited

def get_category(url_home):
    dict_cate = {}
    home = newspaperedited.build(url_home)
    category_urls = home.category_urls()
    return list(set(category_urls))

list_cate = [get_category(url) for url in urls]

In [7]:
list_categorys = []
for i in list_cate:
    list_categorys += i  
list_categorys += urls
list_categorys

['https://vnexpress.net/so-hoa',
 'https://vnexpress.net/giai-tri',
 'https://vnexpress.net/tin-tuc-24h',
 'https://vnexpress.net/oto-xe-may',
 'https://vnexpress.net/du-lich',
 'https://vnexpress.net/',
 'https://vnexpress.net',
 'https://vnexpress.net/thoi-su',
 'https://vnexpress.net/phap-luat',
 'https://vnexpress.net/doi-song',
 'https://vnexpress.net/goc-nhin',
 'https://vnexpress.net/khoa-hoc',
 'https://vnexpress.net/hai',
 'https://vnexpress.net/kinh-doanh',
 'https://vnexpress.net/the-thao',
 'https://video.vnexpress.net',
 'https://vnexpress.net/tam-su',
 'https://vnexpress.net/the-gioi',
 'https://vnexpress.net/suc-khoe',
 'https://vnexpress.net/giao-duc',
 'https://vnexpress.net/y-kien',
 'https://e.vnexpress.net',
 'https://www.24h.com.vn',
 'https://www.24h.com.vn/',
 'http://baogia.24h.com.vn',
 'https://tuoitre.vn/gia-that.htm',
 'https://tuoitre.vn/media.htm',
 'https://tuoitre.vn/phap-luat.htm',
 'https://tuoitre.vn/photo.htm',
 'https://tuoitre.vn',
 'https://tuoitr

## get keywords from url

In [8]:
def get_text_url(url):
    text = ''
    try:
        article = Article(url)
        article.download()
        article.parse()
    except:
        return text, url
    text = article.text.replace('\n', '.\n')
    url = article.url
    return text, url


In [9]:
from pyvi import ViTokenizer
from collections import Counter

def get_keywords_from_text(text):
    tokens = ViTokenizer.tokenize(text)
    tokens = ViTokenizer.spacy_tokenize(tokens)[0]
    tokens = list(filter(lambda x: len(x)>1, tokens))
    counter_tokens = Counter(tokens)
    counter_tokens = dict(counter_tokens)
    counter_tokens = dict(sorted(counter_tokens.items(), key=lambda x:-x[1]))
    return counter_tokens


In [10]:
def get_important_score(dict_keywords, keys):
    score = 0
    for key in dict_keywords.keys():
        for k in keys:
            if k == key.lower():
                score += dict_keywords[key]
    return score


## ranking articles

In [11]:
list_article_url = []

dict_url = {}
for category in list_categorys:
    list_article_url = get_link_articles_from_url(category)

list_article_url = list(set(list_article_url))

In [12]:
len(list_article_url)

1206

In [13]:
# t = get_text_url(list_article_url[100])
# print(list_article_url[100])
# print(t)

In [14]:
list_text_url = []
cnt = 0
for article_url in list_article_url:
    text, url = get_text_url(article_url)
    if text != '':
        list_text_url.append((text, url))
    else:
        continue

    if cnt%10==0:
        print(len(list_text_url), end='-')
    cnt += 1

1-11-21-31-41-51-61-71-81-91-101-111-121-131-141-151-161-171-181-191-201-211-221-231-241-251-261-271-281-291-301-311-321-331-341-351-361-371-381-391-401-411-421-431-441-451-461-471-481-491-501-511-521-531-541-551-561-571-581-591-601-611-621-631-641-651-661-671-681-691-701-711-721-731-741-751-761-771-781-791-801-811-821-831-841-851-861-871-881-891-901-911-921-931-941-951-961-971-981-991-1001-1011-1021-1031-1041-1051-1061-1071-1081-1091-1101-1111-1121-1131-

In [15]:
print(len(list_text_url))

1136


In [16]:
dict_index_and_score = {i:0 for i in range(len(list_text_url))}
# dict_index_and_score {0:0, 1:0, 2:0, .....}

In [17]:
list_keys = []
cnt = 0
for text, url in list_text_url:
    keys = get_keywords_from_text(text)
    dict_index_and_score[cnt] = get_important_score(keys, keywords)
    if cnt%10==0:
        print(cnt, end='-')
    cnt += 1


0-10-20-30-40-50-60-70-80-90-100-110-120-130-140-150-160-170-180-190-200-210-220-230-240-250-260-270-280-290-300-310-320-330-340-350-360-370-380-390-400-410-420-430-440-450-460-470-480-490-500-510-520-530-540-550-560-570-580-590-600-610-620-630-640-650-660-670-680-690-700-710-720-730-740-750-760-770-780-790-800-810-820-830-840-850-860-870-880-890-900-910-920-930-940-950-960-970-980-990-1000-1010-1020-1030-1040-1050-1060-1070-1080-1090-1100-1110-1120-1130-

In [18]:
list_index = dict(sorted(dict_index_and_score.items(), key=lambda x : -x[1]))

In [19]:
cnt = 0 
ans = []
for i in list_index:
    cnt+=1 
    print(i, list_index[i])
    ans.append(list_text_url[i][1])
    print(list_text_url[i][1])
    if cnt == 20:
        break

119 15
https://vnexpress.net/nhung-diem-moi-trong-cac-quyet-dinh-nhan-su-cua-quoc-hoi-4260666.html
1087 15
https://vnexpress.net/ket-qua-phe-chuan-14-tan-thanh-vien-chinh-phu-4260677.html
17 9
https://vnexpress.net/chinh-phu-moi-4256811.html
376 9
https://vnexpress.net/chien-dich-tiem-vaccine-covid-19-cua-thai-lan-bi-chi-trich-4260629.html
738 8
https://vnexpress.net/tan-bo-truong-nguyen-van-hung-ho-chieu-vaccine-la-chia-khoa-mo-cua-4260489.html
191 7
https://vnexpress.net/thu-tuong-anh-nhuong-ghe-du-tang-le-hoang-than-philip-4261147.html
1091 7
https://vnexpress.net/gan-137-trieu-ca-toan-cau-campuchia-nguy-co-vo-tran-covid-19-4261377.html
510 5
https://vnexpress.net/quyen-luc-mem-cua-chu-tich-nuoc-4259594.html
808 5
https://vnexpress.net/chinh-phu-huu-hieu-4257910.html
465 4
https://vnexpress.net/so-ca-covid-19-theo-ngay-o-thai-lan-cao-ky-luc-4261453.html
880 4
https://www.24h.com.vn/infographics-c765.html
43 3
https://vnexpress.net/covid-19-tai-chau-a-nong-tro-lai-4260253.html
222 3


## compare data

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances

tfidf_vectorizer = TfidfVectorizer()
all_text_tfidf = tfidf_vectorizer.fit_transform(list_text_url[:20][0])

list_compare = []

for i in range(20):
    for j in range(i+1, 20):
        tf_idf_i = tfidf_vectorizer.transform(list_text_url[i][0])
        tf_idf_j = tfidf_vectorizer.transform(list_text_url[j][0])
        dist = cosine_distances(tf_idf_i, tf_idf_j)
        list_compare.append((dist, i, j))

list_compare = list(sorted(list_compare, key=lambda x: -x[0]))
list_compare


ValueError: Iterable over raw text documents expected, string object received.

## export data

In [ ]:
path = 'datacrawled/'
cnt = 0
dict_title = {}
for url in ans:
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    text = article.text.replace('\n', '.\n')
    title = article.title
    summary = article.summary
    url = article.url
    if title not in dict_title.keys():
        dict_title[title] = 1
        cnt += 1
        with open(path+'article'+str(cnt)+'.txt', 'w') as f:
            f.write(title+'\n')
            f.write(url+'\n')
            f.write(summary+'\n')